In [1]:
import dash
from dash import Dash
from dash import dcc
from dash import html
from dash import dash_table
from dash.dependencies import Input, Output, State
from dash.exceptions import PreventUpdate

import plotly.express as px
import datetime as dt
import time
import numpy as np

In [2]:
tab_style = {
    'borderBottom': '1px solid #d6d6d6',
    'padding': '6px',
    'font-size': '30px',
    'fontWeight': 'bold'
}

tab_selected_style = {
    'borderTop': '1px solid #d6d6d6',
    'borderBottom': '1px solid #d6d6d6',
    'backgroundColor': '#000091',
    'color': 'white',
    'padding': '6px',
    'font-size': '30px'
}

header = {
    'font-size': '25px',
    'fontWeight': 'bold'
}

searchBox = {
    'width': '100%',
    'height': '200px',
    'margin-top': '-802px',
    'margin-left': '320px',
    'borderWidth': '1px',
    'borderStyle': 'solid',
    'borderRadius': '5px',
    'borderColor': '#000091',
    'padding-left': '10px'
}

In [3]:
app = Dash(__name__)
app.layout = html.Div(children=[
    html.H1(children=['Rent-A-Bike DBMS'], style=header),
    html.Div(id='home', className='control_tabs',children=[
        dcc.Tabs(id='view_tabs',value='bookings', children=[
            dcc.Tab(
                label='Bookings',
                value='bookings',
                style=tab_style,
                selected_style=tab_selected_style
            ),
            dcc.Tab(
                label='HR',
                value='hr',
                style=tab_style,
                selected_style=tab_selected_style
            )
        ])
    ])
])

In [4]:
if __name__ == '__main__':
    app.run_server(debug=True, host="localhost", port="8005")

Dash is running on http://localhost:8005/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


/Users/dennisknierim/miniconda3/lib/python3.8/site-packages/traitlets/traitlets.py:2202: FutureWarning: Supporting extra quotes around strings is deprecated in traitlets 5.0. You can use 'hmac-sha256' instead of '"hmac-sha256"' if you require traitlets >=5.
  warn(
/Users/dennisknierim/miniconda3/lib/python3.8/site-packages/traitlets/traitlets.py:2157: FutureWarning: Supporting extra quotes around Bytes is deprecated in traitlets 5.0. Use 'a83c4195-e0dd-415e-80f8-5b9fe825d920' instead of 'b"a83c4195-e0dd-415e-80f8-5b9fe825d920"'.
  warn(
Traceback (most recent call last):
  File "/Users/dennisknierim/miniconda3/lib/python3.8/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/dennisknierim/miniconda3/lib/python3.8/site-packages/traitlets/config/application.py", line 845, in launch_instance
    app.initialize(argv)
  File "/Users/dennisknierim/miniconda3/lib/python3.8/site-packages/traitlets/config/application.py", line 88, in inner
  

SystemExit: 1

/Users/dennisknierim/miniconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3369: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
